In [2]:
# config.py
# we define all the configuration here
import pickle

with open('multi_class_dict_746.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

NB_FAMILIES = 736
MAX_LEN = 100
TRAIN_BATCH_SIZE = 256 #16
VALID_BATCH_SIZE = 128 #8
EPOCHS = 10
CHAR_INDEX_DICT = {'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'k': 9,
 'l': 10,
 'm': 11,
 'n': 12,
 'o': 13,
 'p': 14,
 'q': 15,
 'r': 16,
 's': 17,
 't': 18,
 'u': 19,
 'v': 20,
 'w': 21,
 'x': 22,
 'y': 23,
 'z': 24}
MULTI_CLASS_DICT = loaded_dict

In [3]:
# dataset.py
import torch
class PFAMDataset:
    def __init__(self, reviews, targets):
         """
         :param reviews: this is a numpy array
         :param targets: a vector, numpy array
         """
         self.reviews = reviews
         self.target = targets
    def __len__(self):
        # returns length of the dataset
        return len(self.reviews)

    def __getitem__(self, item):
         # for any given item, which is an int,
         # return review and targets as torch tensor
         # item is the index of the item in concern
         review = self.reviews[item, :]
         target = self.target[item]
         return {"review": torch.tensor(review, dtype=torch.long),
                 "target": torch.tensor(target, dtype=torch.float) }

In [4]:
# engine.py

import torch
import torch.nn as nn

def train(data_loader, model, optimizer, device):
     """
    This is the main training function that trains model
     for one epoch
     :param data_loader: this is the torch dataloader
     :param model: model (lstm model)
     :param optimizer: torch optimizer, e.g. adam, sgd, etc.
     :param device: this can be "cuda" or "cpu"
     """

     # set model to training mode
     model.train()
     # go through batches of data in data loader
     for data in data_loader:
         # fetch review and target from the dict
         reviews = data["review"]
         targets = data["target"]
         # move the data to device that we want to use
         reviews = reviews.to(device, dtype=torch.long)
         targets = targets.to(device, dtype=torch.float)
         # clear the gradients
         optimizer.zero_grad()
         # make predictions from the model
         predictions = model(reviews)
         # calculate the loss
         loss = nn.CrossEntropyLoss()(
         predictions,
         targets.view(-1, 1)
         )
         # compute gradient of loss w.r.t.
         # all parameters of the model that are trainable
         loss.backward()
         # single optimization step
         optimizer.step()

def evaluate(data_loader, model, device):
     # initialize empty lists to store predictions
     # and targets
     final_predictions = []
     final_targets = []
     # put the model in eval mode
     model.eval()
     # disable gradient calculation
     with torch.no_grad():
         for data in data_loader:
             reviews = data["review"]
             targets = data["target"]
             reviews = reviews.to(device, dtype=torch.long)
             targets = targets.to(device, dtype=torch.float)
             # make predictions
             predictions = model(reviews)
             # move predictions and targets to list
             # we need to move predictions and targets to cpu too
             predictions = predictions.cpu().numpy().tolist()
             targets = data["target"].cpu().numpy().tolist()
             final_predictions.extend(predictions)
             final_targets.extend(targets)
     # return final predictions and targets
     return final_predictions, final_targets

In [5]:
# lstm.py
import torch
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, embedding_matrix):
         """
         :param embedding_matrix: numpy array with vectors for all words
         """
         super(LSTM, self).__init__()
         # number of words = number of rows in embedding matrix
         num_words = embedding_matrix.shape[0]
         # dimension of embedding is num of columns in the matrix
         embed_dim = embedding_matrix.shape[1]
         # we define an input embedding layer
         self.embedding = nn.Embedding(
         num_embeddings=num_words,
         embedding_dim=embed_dim
         )
         # embedding matrix is used as weights of
         # the embedding layer
         self.embedding.weight = nn.Parameter(
         torch.tensor(
         embedding_matrix,
         dtype=torch.float32
         )
         )
         # we dont want to train the pretrained embeddings
         self.embedding.weight.requires_grad = False
         # a simple bidirectional LSTM with
         # hidden size of 128
         self.lstm = nn.LSTM(
             embed_dim,
             128,
             bidirectional=True,
             batch_first=True,
         )
         # output layer which is a linear layer
         # we have only one output
         # input (512) = 128 + 128 for mean and same for max pooling
         self.out = nn.Linear(512, NB_FAMILIES)

    def forward(self, x):

         # pass data through embedding layer
         # the input is just the tokens
         x = self.embedding(x)
         # move embedding output to lstm
         x, _ = self.lstm(x)
         # apply mean and max pooling on lstm output
         avg_pool = torch.mean(x, 1)
         max_pool, _ = torch.max(x, 1)

         # concatenate mean and max pooling
         # this is why size is 512
         # 128 for each direction = 256
         # avg_pool = 256 and max_pool = 256
         out = torch.cat((avg_pool, max_pool), 1)
         # pass through the output layer and return the output
         out = self.out(out)
         # return linear output
         return out

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicConvResBlock(nn.Module):

    def __init__(self, input_dim=128, n_filters=256, kernel_size=3, padding=1, stride=1, shortcut=False, downsample=None):
        super(BasicConvResBlock, self).__init__()

        self.downsample = downsample
        self.shortcut = shortcut

        self.conv1 = nn.Conv1d(input_dim, n_filters, kernel_size=kernel_size, padding=padding, stride=stride)
        self.bn1 = nn.BatchNorm1d(n_filters)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(n_filters, n_filters, kernel_size=kernel_size, padding=padding, stride=stride)
        self.bn2 = nn.BatchNorm1d(n_filters)

    def forward(self, x):

        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.shortcut:
            if self.downsample is not None:
                residual = self.downsample(x)
            out += residual

        out = self.relu(out)

        return out


class VDCNN(nn.Module):

    def __init__(self,embedding_matrix):
        super(VDCNN, self).__init__()

        layers = []
        fc_layers = []
        # number of words = number of rows in embedding matrix
        num_embedding = embedding_matrix.shape[0]
        # dimension of embedding is num of columns in the matrix
        embedding_dim = embedding_matrix.shape[1]

        self.embed = nn.Embedding(num_embedding, num_embedding, padding_idx=0, max_norm=None, norm_type=2, scale_grad_by_freq=False, sparse=False)
        layers.append(nn.Conv1d(embedding_dim, 64, kernel_size=3, padding=1))


        n_conv_block_64, n_conv_block_128, n_conv_block_256, n_conv_block_512 = 2, 2, 2, 2
        n_fc_neurons=2048

        layers.append(BasicConvResBlock(input_dim=64, n_filters=64, kernel_size=3, padding=1))
        for _ in range(n_conv_block_64-1):
            layers.append(BasicConvResBlock(input_dim=64, n_filters=64, kernel_size=3, padding=1))
        layers.append(nn.MaxPool1d(kernel_size=3, stride=2, padding=1)) # l = initial length / 2

        ds = nn.Sequential(nn.Conv1d(64, 128, kernel_size=1, stride=1, bias=False), nn.BatchNorm1d(128))
        layers.append(BasicConvResBlock(input_dim=64, n_filters=128, kernel_size=3, padding=1, downsample=ds))
        for _ in range(n_conv_block_128-1):
            layers.append(BasicConvResBlock(input_dim=128, n_filters=128, kernel_size=3, padding=1))
        layers.append(nn.MaxPool1d(kernel_size=3, stride=2, padding=1)) # l = initial length / 4

        ds = nn.Sequential(nn.Conv1d(128, 256, kernel_size=1, stride=1, bias=False), nn.BatchNorm1d(256))
        layers.append(BasicConvResBlock(input_dim=128, n_filters=256, kernel_size=3, padding=1, downsample=ds))
        for _ in range(n_conv_block_256 - 1):
            layers.append(BasicConvResBlock(input_dim=256, n_filters=256, kernel_size=3, padding=1))
        layers.append(nn.MaxPool1d(kernel_size=3, stride=2, padding=1))

        ds = nn.Sequential(nn.Conv1d(256, 512, kernel_size=1, stride=1, bias=False), nn.BatchNorm1d(512))
        layers.append(BasicConvResBlock(input_dim=256, n_filters=512, kernel_size=3, padding=1, downsample=ds))
        for _ in range(n_conv_block_512 - 1):
            layers.append(BasicConvResBlock(input_dim=512, n_filters=512, kernel_size=3, padding=1))

        layers.append(nn.AdaptiveMaxPool1d(8))
        fc_layers.extend([nn.Linear(8*512, n_fc_neurons), nn.ReLU()])

        fc_layers.extend([nn.Linear(n_fc_neurons, n_fc_neurons), nn.ReLU()])
        fc_layers.extend([nn.Linear(n_fc_neurons, NB_FAMILIES)])

        self.layers = nn.Sequential(*layers)
        self.fc_layers = nn.Sequential(*fc_layers)

        self.__init_weights()

    def __init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight, mode='fan_in')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):

        out = self.embed(x)
        out = out.transpose(1, 2)

        out = self.layers(out)

        out = out.view(out.size(0), -1)

        out = self.fc_layers(out)

        return out

In [9]:
# train.py
import io
import torch
import numpy as np
import pandas as pd
# yes, we use tensorflow
# but not for training the model!
import tensorflow as tf
from sklearn import metrics

def train(data_loader, model, optimizer, device):
     """
    This is the main training function that trains model
     for one epoch
     :param data_loader: this is the torch dataloader
     :param model: model (lstm model)
     :param optimizer: torch optimizer, e.g. adam, sgd, etc.
     :param device: this can be "cuda" or "cpu"
     """

     # set model to training mode
     model.train()
     # go through batches of data in data loader
     for data in data_loader:
         # fetch review and target from the dict
         reviews = data["review"]
         targets = data["target"]
         # move the data to device that we want to use
         reviews = reviews.to(device, dtype=torch.long)
         targets = targets.to(device, dtype=torch.long)
         # clear the gradients
         optimizer.zero_grad()
         # make predictions from the model
         predictions = model(reviews)

         # calculate the loss
         loss = nn.CrossEntropyLoss()(
         predictions,
         targets
         )
         # compute gradient of loss w.r.t.
         # all parameters of the model that are trainable
         loss.backward()
         # single optimization step
         optimizer.step()

def evaluate(data_loader, model, device):
     # initialize empty lists to store predictions
     # and targets
     final_predictions = []
     final_targets = []
     # put the model in eval mode
     model.eval()
     # disable gradient calculation
     with torch.no_grad():
         for data in data_loader:
             reviews = data["review"]
             targets = data["target"]
             reviews = reviews.to(device, dtype=torch.long)
             targets = targets.to(device, dtype=torch.float)
             # make predictions
             predictions = model(reviews)
             # move predictions and targets to list
             # we need to move predictions and targets to cpu too
             predictions = predictions.cpu().numpy().tolist()
             targets = data["target"].cpu().numpy().tolist()
             final_predictions.extend(predictions)
             final_targets.extend(targets)
     # return final predictions and targets
     return final_predictions, final_targets

def create_embedding_matrix(word_index, embedding_dict):
    """
    This function creates the embedding matrix.
    :param word_index: a dictionary with word:index_value
    :param embedding_dict: a dictionary with word:embedding_vector
    :return: a numpy array with embedding vectors for all known words
    """
    # initialize matrix with zeros
    embedding_matrix = []
    # loop over all the words
    for word, i in word_index.items():
        # if word is found in pre-trained embeddings,
        # update the matrix. if the word is not found,
        # the vector is zeros!
        if word in embedding_dict:
            embedding_matrix.append(embedding_dict[word])
        # return embedding matrix
    return np.array(embedding_matrix)


def tokenizing(df, char_index_dict):
    final_string = []
    for sentence in df['sequence']:
        sequence = []
        for word in sentence:
            x = char_index_dict[word]
            sequence.append(str(x))
        final_string.append(sequence)
    df['clean_seq'] = final_string

    return df

def one_hot():
    nb_classes = len(CHAR_INDEX_DICT)
    one_hot_embedding_mat = np.eye(nb_classes)
    one_hot_embedding = {}
    for key, value in enumerate(CHAR_INDEX_DICT):
        one_hot_embedding[value] = one_hot_embedding_mat[key]
    return one_hot_embedding

def run(df, fold):
    """
    Run training and validation for a given fold
    and dataset
    :param df: pandas dataframe with kfold column
    :param fold: current fold, int
    """
    # fetch training dataframe
    train_df = df[df.kfold != fold].reset_index(drop=True)
    # fetch validation dataframe
    valid_df = df[df.kfold == fold].reset_index(drop=True)
    print("Fitting tokenizer")
    # convert training data to sequences
    # for example : "bad movie" gets converted to
    # [24, 27] where 24 is the index for bad and 27 is the
    # index for movie
    xtrain = list(tokenizing(train_df,CHAR_INDEX_DICT)['clean_seq'])
    # similarly convert validation data to
    # sequences
    xtest = list(tokenizing(valid_df,CHAR_INDEX_DICT)['clean_seq'])
    # zero pad the training sequences given the maximum length
    # this padding is done on left hand side
    # if sequence is > MAX_LEN, it is truncated on left hand side too
    xtrain = tf.keras.preprocessing.sequence.pad_sequences(
        xtrain, maxlen=MAX_LEN
    )
    # zero pad the validation sequences
    xtest = tf.keras.preprocessing.sequence.pad_sequences(
        xtest, maxlen=MAX_LEN
    )
    # initialize dataset class for training
    train_dataset = PFAMDataset(
        reviews=xtrain,
        targets=train_df.family_id.values
    )
    # create torch dataloader for training
    # torch dataloader loads the data using dataset
    # class in batches specified by batch size
    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=TRAIN_BATCH_SIZE,
        num_workers=2
    )
    # initialize dataset class for validation
    valid_dataset = PFAMDataset(
        reviews=xtest,
        targets=valid_df.family_id.values
    )

    # create torch dataloader for validation
    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=VALID_BATCH_SIZE,
        num_workers=1
    )
    print("Loading embeddings")
    # load embeddings as shown previously
    embedding_dict = one_hot()
    embedding_matrix = create_embedding_matrix(
        CHAR_INDEX_DICT, embedding_dict
    )
    # create torch device, since we use gpu, we are using cuda
    print(torch.cuda.is_available())
    device = torch.device("cuda")
    # fetch our model
    # model = LSTM(embedding_matrix)
    model = VDCNN(embedding_matrix)
    # send model to device
    model.to(device)

    # initialize Adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    print("Training Model")
    # set best accuracy to zero
    best_accuracy = 0
    # set early stopping counter to zero
    early_stopping_counter = 0
    # train and validate for all epochs
    for epoch in range(EPOCHS):
        # train one epoch
        train(train_data_loader, model, optimizer, device)
        # validate
        outputs, targets = evaluate(
            valid_data_loader, model, device
        )
        # use threshold of 0.5
        # please note we are using linear layer and no sigmoid
        # you should do this 0.5 threshold after sigmoid
        outputs = np.array(outputs) 
        # calculate accuracy
        accuracy = metrics.accuracy_score(targets, np.argmax(outputs,axis=1))
        precision = metrics.precision_score(targets, np.argmax(outputs,axis=1),average='macro')
        recall = metrics.recall_score(targets, np.argmax(outputs,axis=1),average='macro')

        print(
            f"FOLD:{fold}, Epoch: {epoch}, Accuracy Score = {accuracy}, Precision Score = {precision}, Recall Score = {recall}"
        )
        # simple early stopping
        if accuracy > best_accuracy:
            best_accuracy = accuracy
        else:
            early_stopping_counter += 1
        if early_stopping_counter > 2:
            break
    return accuracy,precision, recall

if __name__ == "__main__":
     # load data
     df = pd.read_csv("/content/pfam_736_folds.csv")
     # train for all folds
     accuracy_0,precision_0, recall_0 = run(df, fold=0)
     accuracy_1,precision_1, recall_1 = run(df, fold=1)
     accuracy_2,precision_2, recall_2 = run(df, fold=2)
     accuracy_3,precision_3, recall_3 = run(df, fold=3)
     accuracy_4,precision_4, recall_4 = run(df, fold=4)
     print('Average accuracy : ' + str(np.mean(np.array([accuracy_0,accuracy_1,accuracy_2,accuracy_3,accuracy_4]))))
     print('Average precision : ' + str(np.mean(np.array([precision_0,precision_1,precision_2,precision_3,precision_4]))))
     print('Average recall : ' + str(np.mean(np.array([recall_0,recall_1,recall_2,recall_3,recall_4]))))


Fitting tokenizer
Loading embeddings
True
Training Model
FOLD:0, Epoch: 0, Accuracy Score = 0.516857720836143, Precision Score = 0.3806800471974179, Recall Score = 0.32444074598341166


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


FOLD:0, Epoch: 1, Accuracy Score = 0.699637559002023, Precision Score = 0.633940114596063, Recall Score = 0.5529808799662123


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


FOLD:0, Epoch: 2, Accuracy Score = 0.7962744436952124, Precision Score = 0.7575558275811801, Recall Score = 0.6941085262732394


...

FOLD:4, Epoch: 6, Accuracy Score = 0.8751264329062711, Precision Score = 0.8469977381393384, Recall Score = 0.8050646605022543
FOLD:4, Epoch: 7, Accuracy Score = 0.8335300067430883, Precision Score = 0.8200003853826744, Recall Score = 0.7470035824888012


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


FOLD:4, Epoch: 8, Accuracy Score = 0.8815323668240054, Precision Score = 0.8575186839063172, Recall Score = 0.8173681085013211


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


FOLD:4, Epoch: 9, Accuracy Score = 0.8787508428860418, Precision Score = 0.8556219307614149, Recall Score = 0.8100845748559438
Average accuracy : 0.8816925151719488
Average precision : 0.8526331716342757
Average recall : 0.8179768336895993
